## Merge object classes from different sources into single dataset

In [1]:
cd ../

C:\Users\luisr\Repositories\Flask APIs\ai-camera-client


#### Save yolov8 class names

In [25]:
yolov8_names_path = 'data/yolov8/yolov8.names'

from ultralytics import YOLO

model = YOLO()

results = model.predict()

classes = results[0].names

class_names = [classes[i] for i in sorted(list(classes.keys()))]

with open(yolov8_names_path, 'w') as fw:
    fw.write('\n'.join(class_names))

print('\nYolov8 names saved to:', yolov8_names_path)

WARNING ⚠️ 'source' is missing. Using 'source=C:\Users\luisr\anaconda3\Lib\site-packages\ultralytics\assets'.

image 1/2 C:\Users\luisr\anaconda3\Lib\site-packages\ultralytics\assets\bus.jpg: 640x480 4 persons, 1 bus, 1 stop sign, 122.0ms
image 2/2 C:\Users\luisr\anaconda3\Lib\site-packages\ultralytics\assets\zidane.jpg: 384x640 2 persons, 1 tie, 126.4ms
Speed: 3.0ms preprocess, 124.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

Yolov8 names saved to: data/yolov8/yolov8.names


#### Construct obejct datasets independently for each model

In [2]:
import pandas as pd

yolov4_names_path = 'data/yolov4/coco.names'
yolov4_names = open(yolov4_names_path).read().split('\n')

objects_yolov4 = []
for i in range(len(yolov4_names)):
    obj = {
        'class_id': i,
        'class_name': yolov4_names[i],
        'model_name': 'yolov4',
        'process_name': 'yolov4'
    }
    objects_yolov4.append(obj)    

yolov8_names_path = 'data/yolov8/yolov8.names'
yolov8_names = open(yolov8_names_path).read().split('\n')

objects_yolov8 = []
for i in range(len(yolov8_names)):
    obj = {
        'class_id': i,
        'class_name': yolov8_names[i],
        'model_name': 'yolov8',
        'process_name': 'yolov8'
    }
    objects_yolov8.append(obj)

#### Merge objects from different sources

In [8]:
# Combine objects into single list
objects = objects_yolov4 + objects_yolov8

objects[:5]

[{'class_id': 0,
  'class_name': 'person',
  'model_name': 'yolov4',
  'process_name': 'yolov4'},
 {'class_id': 1,
  'class_name': 'bicycle',
  'model_name': 'yolov4',
  'process_name': 'yolov4'},
 {'class_id': 2,
  'class_name': 'car',
  'model_name': 'yolov4',
  'process_name': 'yolov4'},
 {'class_id': 3,
  'class_name': 'motorbike',
  'model_name': 'yolov4',
  'process_name': 'yolov4'},
 {'class_id': 4,
  'class_name': 'aeroplane',
  'model_name': 'yolov4',
  'process_name': 'yolov4'}]

---
### Manage mongo database

#### Set connection parameters

In [4]:
MONGO_API_URL = 'http://localhost:5000'
db = 'octacity'

#### Get mongo records

In [5]:
import requests

url = f'{MONGO_API_URL}/{db}/objects'

res = requests.get(url)

print('status:', res.status_code)
print('message:', res.reason)

if res.ok:
    objects_mongo = res.json()
    print('\n', objects_mongo[:5])

status: 200
message: OK

 [{'_id': '661f087a58ed01b5246b2bdf', 'class_id': 0, 'class_name': 'person', 'model_name': 'yolov4', 'process_name': 'yolov4'}, {'_id': '661f087a58ed01b5246b2be0', 'class_id': 1, 'class_name': 'bicycle', 'model_name': 'yolov4', 'process_name': 'yolov4'}, {'_id': '661f087a58ed01b5246b2be1', 'class_id': 2, 'class_name': 'car', 'model_name': 'yolov4', 'process_name': 'yolov4'}, {'_id': '661f087a58ed01b5246b2be2', 'class_id': 3, 'class_name': 'motorbike', 'model_name': 'yolov4', 'process_name': 'yolov4'}, {'_id': '661f087a58ed01b5246b2be3', 'class_id': 4, 'class_name': 'aeroplane', 'model_name': 'yolov4', 'process_name': 'yolov4'}]


#### Delte all records in mongo collection

In [6]:
import requests

url = f'{MONGO_API_URL}/{db}/objects'

res = requests.delete(url)

print('status:', res.status_code)
print('message:', res.reason)
print('response:', res.text)

status: 200
message: OK
response: {"deleted_count":182,"message":"All records deleted successfully"}



#### Post objects to mongo collection

In [9]:
collection_name = 'classes'

import requests

url = f'{MONGO_API_URL}/{db}/{collection_name}'
headers = {'Content-Type': 'application/json'}
data = objects.copy()
# data = objects_yolov4.copy()

res = requests.post(url, headers=headers, json=data)

print('status:', res.status_code)
print('message:', res.reason)
print('response:', res.text)

status: 201
message: CREATED
response: {"data":[{"_id":"661f57279e085933a10b57a8","class_id":0,"class_name":"person","model_name":"yolov4","process_name":"yolov4"},{"_id":"661f57279e085933a10b57a9","class_id":1,"class_name":"bicycle","model_name":"yolov4","process_name":"yolov4"},{"_id":"661f57279e085933a10b57aa","class_id":2,"class_name":"car","model_name":"yolov4","process_name":"yolov4"},{"_id":"661f57279e085933a10b57ab","class_id":3,"class_name":"motorbike","model_name":"yolov4","process_name":"yolov4"},{"_id":"661f57279e085933a10b57ac","class_id":4,"class_name":"aeroplane","model_name":"yolov4","process_name":"yolov4"},{"_id":"661f57279e085933a10b57ad","class_id":5,"class_name":"bus","model_name":"yolov4","process_name":"yolov4"},{"_id":"661f57279e085933a10b57ae","class_id":6,"class_name":"train","model_name":"yolov4","process_name":"yolov4"},{"_id":"661f57279e085933a10b57af","class_id":7,"class_name":"truck","model_name":"yolov4","process_name":"yolov4"},{"_id":"661f57279e085933a